In [2]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import fractions


In [191]:
class HouseVotes2DB(object):
    
    def __init__(self):
        pass

    def roll_call_summary(self, year, to_postgres=True):
        url = f"http://clerk.house.gov/evs/{year}"
        rolls = ['ROLL_000.asp'] + [f'ROLL_{i * 100}.asp' for i in range(1, 20)]

        tables = []
        for r in rolls:
            try:
                data = requests.get(os.path.join(url, r))
                soup = BeautifulSoup(data.text)
                tables.append(self._roll_call_table(soup))
            except:
                pass

        df = pd.concat(tables, ignore_index=True)
        df['date'] = str(year) + '-' + df['date']
        df['date'] = pd.to_datetime(df['date'], format='%Y-%d-%b', errors='coerce')
        
        if to_postgres:
            
        
        return df

    def _roll_call_table(self, soup):
        table = soup.find("table")
        rows = table.find_all('tr')
        session_num = re.search('(Congress - )([1-2])', soup.text).group(2)
        dfdata = []
        for row in rows:
            cells = row.find_all('td')
            try:
                link = cells[0].a['href']
                roll = cells[0].text
                date = cells[1].text
                bill = cells[2].text.replace(" ", "").lower()
                congress = "".join([i for i in cells[2].a['href'].split("/")[4] if i.isdigit()])
                typ = cells[3].text
                res = cells[4].text
                title = cells[5].text

                dfdata.append(
                    {'link': link, 
                     'rollcall': roll, 
                     'date': date, 
                     'bill': bill, 
                     'session': session_num,
                     'congress': congress, 
                     'type': typ, 
                     'result': res, 
                     'title': title})
            except:
                pass
        return pd.DataFrame(dfdata)


    def roll_call_single(self, year, roll):
        # 'http://clerk.house.gov/cgi-bin/vote.asp?year=2019&rollnumber=99'
        url = f'http://clerk.house.gov/cgi-bin/vote.asp?year={year}&rollnumber={roll}'
        data = requests.get(url)
        soup = BeautifulSoup(data.text)

        congress = soup.find('congress').text
        session = ''.join([i for i in soup.find('session').text if i.isdigit()])
        chamber = soup.find('chamber').text
        rollcall = soup.find('rollcall-num').text
        vq = soup.find('vote-question').text
        vt = soup.find('vote-type').text
        vr = soup.find('vote-result').text
        date = soup.find('action-date').text
        tofd = soup.find('action-time').text
        title = soup.find('vote-desc').text

        votes = []
        for i in soup.find_all('recorded-vote'):
            votes.append(
                {**i.find('legislator').attrs, **{'vote': i.find('vote').text}}
            )
        df = pd.DataFrame(votes)
        df.columns = [i.replace('-', '_') for i in df.columns]
        df['congress'] = congress
        df['session'] = session
        # df['chamber'] = chamber
        df['rollcall'] = rollcall
        df['vote_question'] = vq
        df['vote_type'] = vt
        df['vote_result'] = vr
        # df['date'] = date
        # df['time'] = tofd
        df['title'] = title

        return df
    
    def 

In [192]:
obj = HouseVotes2DB()
obj.roll_call_summary(2019)

,link,rollcall,date,bill,session,congress,type,result,title
0,http://clerk.house.gov/cgi-bin/vote.asp?year=2...,99,2019-02-27,hr8,1,116,On Passage,P,Bipartisan Background Checks Act
1,http://clerk.house.gov/cgi-bin/vote.asp?year=2...,98,2019-02-27,hr8,1,116,On Motion to Recommit with Instructions,P,Bipartisan Background Checks Act
2,http://clerk.house.gov/cgi-bin/vote.asp?year=2...,97,2019-02-27,hr8,1,116,On Agreeing to the Amendment,A,Horn of Oklahoma Part A Amendment No. 3
3,http://clerk.house.gov/cgi-bin/vote.asp?year=2...,96,2019-02-27,hr8,1,116,On Agreeing to the Amendment,F,Lesko of Arizona Part A Amendment No. 1
4,http://clerk.house.gov/cgi-bin/vote.asp?year=2...,95,2019-02-26,s47,1,116,On Motion to Suspend the Rules and Pass,P,Natural Resources Management Act
...,...,...,...,...,...,...,...,...,...
677,http://clerk.house.gov/cgi-bin/vote.asp?year=2...,602,2019-10-30,hr1373,1,116,On Passage,P,Grand Canyon Centennial Protection Act
678,http://clerk.house.gov/cgi-bin/vote.asp?year=2...,601,2019-10-30,hr1373,1,116,On Motion to Recommit with Instructions,F,Grand Canyon Centennial Protection Act
679,http://clerk.house.gov/cgi-bin/vote.asp?year=2...,600,2019-10-30,hr1373,1,116,On Agreeing to the Amendment,F,Gosar of Arizona Part C Amendment No. 3
680,http://clerk.house.gov/cgi-bin/vote.asp?year=2...,701,2019-12-19,hr5430,1,116,On Passage,P,United States-Mexico-Canada Agreement Implemen...


In [193]:
obj.roll_call_single('http://clerk.house.gov/cgi-bin/vote.asp?year=2019&rollnumber=98')

,name_id,sort_field,unaccented_name,party,state,role,vote,congress,session,rollcall,vote_question,vote_type,vote_result,title
0,A000374,Abraham,Abraham,R,LA,legislator,Yea,116,1,98,On Motion to Recommit with Instructions,YEA-AND-NAY,Passed,Bipartisan Background Checks Act
1,A000370,Adams,Adams,D,NC,legislator,Nay,116,1,98,On Motion to Recommit with Instructions,YEA-AND-NAY,Passed,Bipartisan Background Checks Act
2,A000055,Aderholt,Aderholt,R,AL,legislator,Yea,116,1,98,On Motion to Recommit with Instructions,YEA-AND-NAY,Passed,Bipartisan Background Checks Act
3,A000371,Aguilar,Aguilar,D,CA,legislator,Nay,116,1,98,On Motion to Recommit with Instructions,YEA-AND-NAY,Passed,Bipartisan Background Checks Act
4,A000372,Allen,Allen,R,GA,legislator,Yea,116,1,98,On Motion to Recommit with Instructions,YEA-AND-NAY,Passed,Bipartisan Background Checks Act
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,W000827,Wright,Wright,R,TX,legislator,Yea,116,1,98,On Motion to Recommit with Instructions,YEA-AND-NAY,Passed,Bipartisan Background Checks Act
428,Y000062,Yarmuth,Yarmuth,D,KY,legislator,Nay,116,1,98,On Motion to Recommit with Instructions,YEA-AND-NAY,Passed,Bipartisan Background Checks Act
429,Y000065,Yoho,Yoho,R,FL,legislator,Yea,116,1,98,On Motion to Recommit with Instructions,YEA-AND-NAY,Passed,Bipartisan Background Checks Act
430,Y000033,Young,Young,R,AK,legislator,Yea,116,1,98,On Motion to Recommit with Instructions,YEA-AND-NAY,Passed,Bipartisan Background Checks Act


In [185]:
class SenateVotes2DB(object):
    
    def __init__(self):
        pass
    
    def get_congress_year_summary(self, soup):
        header = soup.find(id='legislative_header').find('h1').text
        header = header.lower().split(' ')
        congress_num = ''.join([i for i in header[3] if i.isdigit()])
        year = ''.join([i for i in header[-1] if i.isdigit()])
        return congress_num, year
    
    def roll_call_summary(self, congress):
        # 'https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_101_1.htm'
        dfdata = []
        for session in [1, 2]:
            data = requests.get(
                f'https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_{congress}_{session}.htm'
            )
            soup = BeautifulSoup(data.text)
            congress_num, congress_year = get_congress_year(soup)
            
            table = soup.find('table')
            rows = table.find_all('tr')
            
            for row in rows:
                try:
                    row = row.replace(r'\xa0', ' ')
                except TypeError:
                    pass
                cells = row.find_all('td')
                if not cells:
                    continue

                vote_id = cells[0].text.split(' ')[1]
                try:
                    vote_id = vote_id[:vote_id.index(r'(')].strip()
                except:
                    pass
                result = cells[1].text
                title = cells[2].text
                bill = ''.join([i for i in cells[3].text if i.isalnum()]).lower()
                mo, day = cells[4].text[:3], cells[4].text[-2:]
                date = '-'.join([mo, day])
                dfdata.append({
                    'congress': congress_num,
                    'link': link,
                    'rollcall': vote_id,
                    'result': result,
                    'title': title,
                    'bill': bill,
                    'date': str(congress_year) + '-' + date,
                    'session': session
                })
                
        return pd.DataFrame(dfdata)
    
    def roll_call_single(self, congress, session, rollcall):
        rollcall = str(rollcall)
        pads = 5 - len(rollcall)
        rollcall = (pads * '0') + str(rollcall)
        url = f"https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress={congress}&session={session}&vote={rollcall}"
        # data = requests.get('https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=102&session=1&vote=00280')
        data = requests.get(url)
        soup = BeautifulSoup(data.text)
        
        header = soup.find('section', {'id': 'legislative_header'})
        header = header.text.strip().split(' ')
        header = [i for i in header if i]
        congress_num = ''.join([i for i in header[3] if i.isdigit()])
        session_num = ''.join([i for i in header[6] if i.isdigit()])
        z = soup.find_all('div', {'class': 'contenttext'})
        meta = z[0].text.strip()
        question = meta[0: meta.index('Vote Number: ')].strip().replace('\n', '')[9:]
        vote_num = meta[
            meta.index('Vote Number: '): meta.index('Vote Date: ')
        ].strip().replace('\n', '').replace('\t', '')[len('Vote Number: '):].strip()
        required_majority = int(round(fractions.Fraction(
            meta[meta.index('Required For Majority: '): meta.index('Vote Result: ')][len('Required For Majority: '):].strip()
        ) * 100, 0))
        
        votes = soup.find('span', {'class': 'contenttext'}).text.split('\n')
        votelist = []
        for v in votes:
            if not v: 
                continue
            try:
                paren_index = v.index(r'(') - 1
            except:
                print(v)
            name = v[:paren_index]
            if ',' in name:
                surname = name.split(',')[0].strip()
                firstname = name.split(',')[-1].strip()
            else:
                surname = name.strip()
                firstname = ''

            v = v[paren_index + 1:]
            v = v.split(', ')
            vote = v[-1]
            party = v[0].split('-')[0][1:]
            state = v[0].split('-')[1][:-1]

            votelist.append(
                {'firstname': firstname,
                 'surname': surname,
                 'state': state,
                 'party': party,
                 'vote': vote,
                 'congress': congress_num,
                 'session': session_num,
                 'question': question,
                 'required_majority': required_majority,
                 'rollcall': vote_num
                }
            )
        return pd.DataFrame(votelist)

In [186]:
obj = SenateVotes2DB()
c = obj.roll_call_summary(102)

In [180]:
obj.roll_call_single(102, 1, 280)

,firstname,surname,state,party,vote,congress,session,question,required_majority,rollcall
0,,Adams,WA,D,Nay,102,1,On Passage of the Bill(H.R. 3435),50,280
1,,Akaka,HI,D,Nay,102,1,On Passage of the Bill(H.R. 3435),50,280
2,,Baucus,MT,D,Nay,102,1,On Passage of the Bill(H.R. 3435),50,280
3,,Bentsen,TX,D,Yea,102,1,On Passage of the Bill(H.R. 3435),50,280
4,,Biden,DE,D,Not Voting,102,1,On Passage of the Bill(H.R. 3435),50,280
...,...,...,...,...,...,...,...,...,...,...
95,,Wallop,WY,R,Nay,102,1,On Passage of the Bill(H.R. 3435),50,280
96,,Warner,VA,R,Yea,102,1,On Passage of the Bill(H.R. 3435),50,280
97,,Wellstone,MN,D,Nay,102,1,On Passage of the Bill(H.R. 3435),50,280
98,,Wirth,CO,D,Not Voting,102,1,On Passage of the Bill(H.R. 3435),50,280


In [135]:
src

,congress,rollcall,result,title,bill,date,session
0,102,280,Passed,On Passage of the Bill: H.R. 3435; A bill to p...,hr3435,1991-Nov-27,1
1,102,279,Agreed to,On the Conference Report: S. 543 Conference Re...,s543,1991-Nov-27,1
2,102,278,Rejected,On the Cloture Motion: Motion To Invoke Clotur...,hr3371,1991-Nov-27,1
3,102,277,Agreed to,On the Conference Report: H.R. 2950 Conference...,hr2950,1991-Nov-27,1
4,102,276,Agreed to,On the Amendment S.Amdt. 1443: Levin Amendment...,hr3807,1991-Nov-25,1
...,...,...,...,...,...,...,...
545,102,5,Rejected,On the Amendment S.Amdt. 1476: Hatch Amendment...,s2,1992-Jan-23,2
546,102,4,Rejected,On the Amendment S.Amdt. 1478: Helms Amendment...,s2,1992-Jan-23,2
547,102,3,Agreed to,On the Amendment S.Amdt. 1474: Hatfield Amendm...,s2,1992-Jan-23,2
548,102,2,Agreed to,On the Amendment S.Amdt. 1473: Cochran Amendme...,s2,1992-Jan-23,2


In [46]:
import re
re.search('(Congress-)([1-2])', soup.text).group(2)

'1'

In [51]:
data = requests.get('https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=102&session=1&vote=00280')
soup = BeautifulSoup(data.text)

In [154]:
header = soup.find('section', {'id': 'legislative_header'})
header = header.text.strip().split(' ')
header = [i for i in header if i]
congress_num = ''.join([i for i in header[3] if i.isdigit()])
session_num = ''.join([i for i in header[6] if i.isdigit()])

congress_num, session_num

z = soup.find_all('div', {'class': 'contenttext'})
meta = z[0].text.strip()

question = meta[0: meta.index('Vote Number: ')].strip().replace('\n', '')[9:]
vote_num = meta[
    meta.index('Vote Number: '): meta.index('Vote Date: ')
].strip().replace('\n', '').replace('\t', '')[len('Vote Number: '):].strip()
required_majority = int(round(fractions.Fraction(
    meta[meta.index('Required For Majority: '): meta.index('Vote Result: ')][len('Required For Majority: '):].strip()
) * 100, 0))

In [155]:
question, vote_num, required_majority

(' On Passage of the Bill(H.R. 3435)', '280', 50)

In [ ]:
meta

In [168]:
votes = soup.find('span', {'class': 'contenttext'}).text.split('\n')
votelist = []
for v in votes:
    if not v: 
        continue
    try:
        paren_index = v.index(r'(') - 1
    except:
        print(v)
    name = v[:paren_index]
    if ',' in name:
        surname = name.split(',')[0].strip()
        firstname = name.split(',')[-1].strip()
    else:
        surname = name.strip()
        firstname = ''
        
    v = v[paren_index + 1:]
    v = v.split(', ')
    vote = v[-1]
    party = v[0].split('-')[0][1:]
    state = v[0].split('-')[1][:-1]
    
    # print(firstname, surname, vote, party, state)
    votelist.append(
        {'firstname': firstname,
         'surname': surname,
         'state': state,
         'party': party,
         'vote': vote,
         'congress': congress_num,
         'session': session_num,
         'question': question,
         'required_majority': required_majority,
         'rollcall': vote_num
         })
    # break

In [169]:
pd.DataFrame(votelist)

,firstname,surname,state,party,vote,congress,session,question,required_majority,rollcall
0,,Adams,WA,D,Nay,102,1,On Passage of the Bill(H.R. 3435),50,280
1,,Akaka,HI,D,Nay,102,1,On Passage of the Bill(H.R. 3435),50,280
2,,Baucus,MT,D,Nay,102,1,On Passage of the Bill(H.R. 3435),50,280
3,,Bentsen,TX,D,Yea,102,1,On Passage of the Bill(H.R. 3435),50,280
4,,Biden,DE,D,Not Voting,102,1,On Passage of the Bill(H.R. 3435),50,280
...,...,...,...,...,...,...,...,...,...,...
95,,Wallop,WY,R,Nay,102,1,On Passage of the Bill(H.R. 3435),50,280
96,,Warner,VA,R,Yea,102,1,On Passage of the Bill(H.R. 3435),50,280
97,,Wellstone,MN,D,Nay,102,1,On Passage of the Bill(H.R. 3435),50,280
98,,Wirth,CO,D,Not Voting,102,1,On Passage of the Bill(H.R. 3435),50,280
